In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
file_path = "datasets\datos_CB.csv"
student_data = pd.read_csv(file_path)
student_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  97341 non-null  int64  
 1   Unnamed: 0    97341 non-null  int64  
 2   danho         97341 non-null  int64  
 3   ciclo         97341 non-null  int64  
 4   Cod.Asign     97341 non-null  int64  
 5   Asignatura    97341 non-null  object 
 6   Cod.Car.Sec   97341 non-null  object 
 7   Cod.Curso     97341 non-null  int64  
 8   Convocatoria  97341 non-null  int64  
 9   Anho          97341 non-null  int64  
 10  Semestre      97341 non-null  int64  
 11  Aprobado      97341 non-null  int64  
 12  Anho.Firma    97341 non-null  int64  
 13  Primer.Par    97341 non-null  int64  
 14  Segundo.Par   97341 non-null  int64  
 15  AOT           97341 non-null  int64  
 16  Primer.Rec    97341 non-null  int64  
 17  Segundo.Rec   97341 non-null  int64  
 18  Nota.Final    55096 non-nu

In [3]:
columns = ['Unnamed: 0.1', 'Unnamed: 0', 'Asis', '4P_60']
if columns[0] in student_data:
    student_data.drop(columns,axis=1, inplace= True)
student_data.head()

,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
0,2012,1,5012,CALCULO 3,CGF-175-09,3,1,2012,1,0,0,0,0,0,0,0,NaN,es_253
1,2012,1,5015,FISICA 4,CIV-PLS09,4,1,2012,1,0,2012,19,13,9,0,0,NaN,es_254
2,2012,1,5015,FISICA 4,CGF-PLS09,4,1,2012,1,1,2012,17,24,10,0,0,['3'],es_255
3,2012,1,5015,FISICA 4,CIV-PLS09,4,1,2012,1,0,0,0,0,0,0,0,NaN,es_256
4,2012,1,5015,FISICA 4,CIV-PLS09,4,1,2012,1,0,0,16,12,8,0,0,NaN,es_257


In [4]:
#todo esto puede ser un diccionario con numpy unique y queda re loco
course = "ESTATICA"
condition = student_data["Asignatura"] == course
course_data = student_data[condition]
course_data.head()

,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
5258,2012,2,5014,ESTATICA,IND-PLS09,3,1,2012,2,0,0,2,13,0,0,0,NaN,es_469
5259,2012,2,5014,ESTATICA,IND-PLS09,3,1,2012,2,0,0,6,11,0,0,0,NaN,es_470
5260,2012,2,5014,ESTATICA,ELE-PLS09,3,1,2012,2,1,2012,12,24,0,0,0,['2'],es_175
5261,2012,2,5014,ESTATICA,ELE-PLS09,3,1,2012,2,0,0,1,1,0,0,0,NaN,es_910
5262,2012,2,5014,ESTATICA,CIV-PLS09,3,1,2012,2,0,0,0,0,0,0,0,NaN,es_1475


In [5]:
%%capture
print(np.unique(course_data["danho"]))
print(np.unique(course_data["Anho"]))

In [6]:
course_data["Primer.Par"].describe()

count    4824.000000
mean        9.337894
std         5.849116
min         0.000000
25%         5.000000
50%        10.000000
75%        14.000000
max        24.000000
Name: Primer.Par, dtype: float64

In [7]:
approved = len(course_data[course_data["Aprobado"]=="S"])
not_approved = len(course_data[course_data["Aprobado"] == "N"])
total = len(course_data["Aprobado"])
print(f"Approved: {approved}\nNot approved: {not_approved}\nTotal: {total}")

Approved: 0
Not approved: 0
Total: 4824


In [8]:
df_train, df_test= train_test_split(course_data, test_size=0.2, random_state=0)
print(df_train.shape)
print(df_test.shape)


# Saving our training and testing data sets
'''
# Training Set
train_save_path = 'student_train.csv'
train_df.to_csv(train_save_path, index=False)

# Testing Set
test_save_path = os.path.join(os.getcwd(), 'student', 'student-por-test.csv')
test_df.to_csv(test_save_path, index=False)
'''

(3859, 18)
(965, 18)


"\n# Training Set\ntrain_save_path = 'student_train.csv'\ntrain_df.to_csv(train_save_path, index=False)\n\n# Testing Set\ntest_save_path = os.path.join(os.getcwd(), 'student', 'student-por-test.csv')\ntest_df.to_csv(test_save_path, index=False)\n"

In [9]:
# Dummy Encoding the Data
# Coverting categorical data to numerical dummy variables
#train_dummies_df = pd.get_dummies(df_train)
#test_dummies_df = pd.get_dummies(df_test)

# Separating training data into predictors and predictand
use_cols = [column for column in df_train.columns if column not in ['Aprobado']]
train_x = df_train[use_cols]
train_y = df_train['Aprobado']

# Separating testing data into predictors and predictand
test_x = df_test[use_cols]
test_y = df_test['Aprobado']

In [10]:
train_x.head()

,danho,ciclo,Cod.Asign,Asignatura,Cod.Car.Sec,Cod.Curso,Convocatoria,Anho,Semestre,Anho.Firma,Primer.Par,Segundo.Par,AOT,Primer.Rec,Segundo.Rec,Nota.Final,id_anony
5757,2012,2,5014,ESTATICA,ELE-PLS09,3,1,2012,2,2011,13,13,0,0,0,['1'],es_660
58811,2017,2,13015,ESTATICA,MCT-PLS13,3,1,2017,2,0,9,0,2,0,0,NaN,es_2842
59840,2017,2,5014,ESTATICA,MEC-PLS09,3,1,2017,2,0,12,7,3,6,14,NaN,es_1613
46862,2016,2,13015,ESTATICA,ELE-PLS13,3,1,2016,2,0,5,10,2,0,0,NaN,es_2182
36301,2015,2,13015,ESTATICA,CIV-PLS13,3,1,2015,2,2015,9,24,3,0,0,['2'],es_2354


In [11]:
train_y.head()

5757     0
58811    0
59840    0
46862    0
36301    1
Name: Aprobado, dtype: int64

In [ ]:
# Creating the logistic regression CV object.  Using the 'l1' penalty, settting the number
# of folds to 5, and the solver to 'liblinear'
logRegCVSDNo0s = LogisticRegression()

# Fitting the model
logRegCVSDNo0s.fit(train_x_standardized_no0s, train_y_no0s)

# Plotting the CV Accuracy with respect to C values
fig, axs = plt.subplots(1, 1, figsize=(14, 7))
axs.plot(logRegCVSDNo0s.Cs_, logRegCVSDNo0s.scores_[1].mean(axis=0), marker='o')
axs.set_xscale('log')
axs.set_title('Cross Validation Accuracy With Respect to C values', fontsize=18)
axs.set_xlabel('C Values')
axs.set_ylabel('Accuracy')
axs.grid()
print(logRegCVSDNo0s.scores_[1].mean(axis=0).max())
print(logRegCVSDNo0s.C_)



logRegSDAccuracyNo0s = accuracy_score(logRegCVSDNo0s.predict(test_x_standardized), test_y)